In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from transformers import ViTFeatureExtractor, ViTModel, ViTConfig, DistilBertModel, DistilBertConfig
from tqdm.notebook import tqdm
from torch.autograd import Variable
from datetime import datetime, timedelta
import time
import models

## Data Loading

Here the data is loaded and scaled

In [2]:
X_train = np.load('./data/X_train_surge_new.npz')
Y_train = pd.read_csv('./data/Y_train_surge.csv')
X_test = np.load('./data/X_test_surge_new.npz')

# train
slp_train = X_train['slp']
t_slp_train = X_train['t_slp']

t_surge1_input_train = X_train['t_surge1_input']
t_surge2_input_train = X_train['t_surge2_input']

surge1_input_train = X_train['surge1_input']
surge2_input_train = X_train['surge2_input']

mean_surge1_input_train = np.mean(surge1_input_train, axis=1)
std_surge1_input_train = np.std(surge1_input_train, axis=1)
mean_surge2_input_train = np.mean(surge2_input_train, axis=1)
std_surge2_input_train = np.std(surge2_input_train, axis=1)

scaled_surge1_input_train = (surge1_input_train - mean_surge1_input_train[:,None]) / std_surge1_input_train[:,None]
scaled_surge2_input_train = (surge2_input_train - mean_surge2_input_train[:,None]) / std_surge2_input_train[:,None]

t_surge1_output_train = X_train['t_surge1_output']
t_surge2_output_train = X_train['t_surge2_output']

# test
slp_test = X_test['slp']
t_slp_test = X_test['t_slp']

t_surge1_input_test = X_test['t_surge1_input']
t_surge2_input_test = X_test['t_surge2_input']

surge1_input_test = X_test['surge1_input']
surge2_input_test = X_test['surge2_input']

mean_surge1_input_test = np.mean(surge1_input_test, axis=1)
std_surge1_input_test = np.std(surge1_input_test, axis=1)
mean_surge2_input_test = np.mean(surge2_input_test, axis=1)
std_surge2_input_test = np.std(surge2_input_test, axis=1)

scaled_surge1_input_test = (surge1_input_test - mean_surge1_input_test[:,None]) / std_surge1_input_test[:,None]
scaled_surge2_input_test = (surge2_input_test - mean_surge2_input_test[:,None]) / std_surge2_input_test[:,None]

t_surge1_output_test = X_test['t_surge1_output']
t_surge2_output_test = X_test['t_surge2_output']

Now we need to divide the output

In [3]:
Y_1 = Y_train[['surge1_t0', 'surge1_t1', 'surge1_t2', 'surge1_t3', 'surge1_t4', 'surge1_t5', 'surge1_t6', 'surge1_t7', 'surge1_t8', 'surge1_t9']].to_numpy()
Y_2 = Y_train[['surge2_t0', 'surge2_t1', 'surge2_t2', 'surge2_t3', 'surge2_t4', 'surge2_t5', 'surge2_t6', 'surge2_t7', 'surge2_t8', 'surge2_t9']].to_numpy()

Code to have a series of pressures the same as the surges

In [4]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

In [5]:
pressures_same_time_1 = np.empty((*(t_surge1_input_train.shape), 41, 41))
for i, time_series in enumerate(t_surge1_input_train):
    for j, time in enumerate(time_series):
        idx = find_nearest(t_slp_train[i,:].flatten(), time)
        pressures_same_time_1[i, j, :, :] = slp_train[i, idx, :, :]

pressures_same_time_2 = np.empty((*(t_surge2_input_train.shape), 41, 41))
for i, time_series in enumerate(t_surge2_input_train):
    for j, time in enumerate(time_series):
        idx = find_nearest(t_slp_train[i,:].flatten(), time)
        pressures_same_time_2[i, j, :, :] = slp_train[i, idx, :, :]

In [6]:
mean_pressures_same_time_1 = np.mean(pressures_same_time_1, axis=(1,2,3))
std_pressures_same_time_1 = np.std(pressures_same_time_1, axis=(1,2,3))
mean_pressures_same_time_2 = np.mean(pressures_same_time_2, axis=(1,2,3))
std_pressures_same_time_2 = np.std(pressures_same_time_2, axis=(1,2,3))

scaled_pressures_same_time_1 = (pressures_same_time_1 - mean_pressures_same_time_1[:,None, None, None]) / std_pressures_same_time_1[:,None, None, None]
scaled_pressures_same_time_2 = (pressures_same_time_2 - mean_pressures_same_time_2[:,None, None, None]) / std_pressures_same_time_2[:,None, None, None]


In [7]:
def hour_rounder(t):
    return (t.replace(second=0, microsecond=0, minute=0, hour=t.hour) + timedelta(hours=t.minute//30))

In [8]:
def time_to_hour(array):
    hours_array = np.empty_like(array)
    for i, times in enumerate(array):
        for j, t in enumerate(times):
            if t<0:
                tt = (datetime(1970,1,1) + timedelta(seconds=int(t))).timetuple()
            else:
                tt = (datetime.fromtimestamp(int(t))).timetuple() 
            hours_array[i][j] = tt.tm_yday * 24 + tt.tm_hour
    return hours_array / (366 * 24)

In [9]:
hours_in_year_surge_1_train = time_to_hour(t_surge1_input_train)
hours_in_year_surge_2_train = time_to_hour(t_surge2_input_train)
hours_in_year_surge_1_test = time_to_hour(t_surge1_input_test)
hours_in_year_surge_2_test = time_to_hour(t_surge1_input_test)
hours_in_year_surge_1_output_train = time_to_hour(t_surge1_output_train)
hours_in_year_surge_2_output_train = time_to_hour(t_surge2_output_train)
hours_in_year_slp_train = time_to_hour(t_slp_train)
hours_in_year_slp_test = time_to_hour(t_slp_test)

hours_in_year_surge_1_train.shape

datalen = len(surge1_input_train)
trainlen = int(0.9 * datalen)
vallen = datalen - trainlen
train_idx, val_idx = torch.utils.data.random_split(np.arange(datalen), [trainlen, vallen])

pressure1_train, pressure1_val = pressures_same_time_1[train_idx], pressures_same_time_1[val_idx]
surge1_train, surge1_val = surge1_input_train[train_idx], surge1_input_train[val_idx]
t_surge1_train, t_surge1_val = hours_in_year_surge_1_train[train_idx], hours_in_year_surge_1_train[val_idx]
Y_1_train, Y_1_val = Y_1[train_idx], Y_1[val_idx]

train_data = list(zip(pressure1_train, surge1_train, t_surge1_train, Y_1_train))
val_data = list(zip(pressure1_val, surge1_val, t_surge1_val, Y_1_val))

batch_size = 8

train_dataloader = DataLoader(
    train_data,
    batch_size=batch_size,
    shuffle=True
)

val_dataloader = DataLoader(
    val_data,
    batch_size=batch_size,
    shuffle=False
)

nnn = 100
pressures_same_time_1_flatten = pressures_same_time_1.reshape(-1, 1, 41, 41)[:nnn]
surge1_input_train_flatten = surge1_input_train.reshape(-1, 1)[:nnn]
hours_in_year_surge_1_train_flatten = hours_in_year_surge_1_train.reshape(-1, 1)[:nnn]
Y_1_flatten = Y_1.reshape(-1, 1)[:nnn]

datalen = len(pressures_same_time_1_flatten)
trainlen = int(0.9 * datalen)
vallen = datalen - trainlen
train_idx, val_idx = torch.utils.data.random_split(np.arange(datalen), [trainlen, vallen])

pressure1_train, pressure1_val = pressures_same_time_1_flatten[train_idx], pressures_same_time_1_flatten[val_idx]
surge1_train, surge1_val = surge1_input_train_flatten[train_idx], surge1_input_train_flatten[val_idx]
t_surge1_train, t_surge1_val = hours_in_year_surge_1_train_flatten[train_idx], hours_in_year_surge_1_train_flatten[val_idx]
Y_1_train, Y_1_val = Y_1_flatten[train_idx], Y_1_flatten[val_idx]

train_data = list(zip(pressure1_train, surge1_train, t_surge1_train, Y_1_train))
val_data = list(zip(pressure1_val, surge1_val, t_surge1_val, Y_1_val))

batch_size = 8

train_dataloader_flatten = DataLoader(
    train_data,
    batch_size=batch_size,
    shuffle=True
)

val_dataloader_flatten = DataLoader(
    val_data,
    batch_size=batch_size,
    shuffle=False
)

## Data Training

In [10]:
w = torch.linspace(1, 0.1, 10)[np.newaxis]

def benchmark_weighted_losses(output, target):
    loss = torch.mean(w * (output - target)**2)
    return loss

### Transformers

Training of the transformer to compute the features from the pressure

model = PressureEncorder()
# device = torch.device('cuda')
# model = model.to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

epochs = 10

for epoch in range(epochs):
    model.train()
    for x1, x2, x3, y in tqdm(train_dataloader_flatten, total = len(train_dataloader), leave=False):
        # x1, x2, x3, y = x1.to(device), x2.to(device), x3.to(device), y.to(device)
        x1 = x1.type(torch.FloatTensor)
        x2 = x2.type(torch.FloatTensor)
        x3 = x3.type(torch.FloatTensor)
        y = y.type(torch.FloatTensor)
        optimizer.zero_grad()
        pred = model((x1, x2, x3))
        loss = criterion(pred, y)
        loss.backward()
        optimizer.step()
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for x1, x2, x3, y in tqdm(val_dataloader_flatten, total = len(val_dataloader), leave = False):
            # x1, x2, x3, y = x1.to(device), x2.to(device), x3.to(device), y.to(device)
            x1 = x1.type(torch.FloatTensor)
            x2 = x2.type(torch.FloatTensor)
            x3 = x3.type(torch.FloatTensor)
            y = y.type(torch.FloatTensor)
            pred = model((x1, x2, x3))
            loss = criterion(pred, y)
            val_loss += loss.item()
    val_loss /= (len(val_dataloader)*batch_size)
    print(f'Epoch {epoch+1}: Validation Loss = {val_loss}')

### Seq2Seq

In [11]:
nnn = 200

pressures_same_time_1_flatten = scaled_pressures_same_time_1[:nnn]
pressures_same_time_2_flatten = scaled_pressures_same_time_2[:nnn]

surge1_input_train_flatten = surge1_input_train[:nnn]
surge2_input_train_flatten = surge2_input_train[:nnn]

hours_in_year_surge_1_train_flatten = hours_in_year_surge_1_train[:nnn]
hours_in_year_surge_2_train_flatten = hours_in_year_surge_2_train[:nnn]

Y_1_flatten = Y_1[:nnn]
Y_2_flatten = Y_2[:nnn]

mean_pressures_same_time_1_flatten = mean_pressures_same_time_1[:nnn]
std_pressures_same_time_1_flatten = std_pressures_same_time_1[:nnn]
mean_pressures_same_time_2_flatten = mean_pressures_same_time_2[:nnn]
std_pressures_same_time_2_flatten = std_pressures_same_time_2[:nnn]

mean_surge1_input_train_flatten = mean_surge1_input_train[:nnn]
std_surge1_input_train_flatten = std_surge1_input_train[:nnn]
mean_surge2_input_train_flatten = mean_surge2_input_train[:nnn]
std_surge2_input_train_flatten = std_surge2_input_train[:nnn]

In [12]:
datalen = len(pressures_same_time_1_flatten)
trainlen = int(0.9 * datalen)
vallen = datalen - trainlen
train_idx, val_idx = torch.utils.data.random_split(np.arange(datalen), [trainlen, vallen])

pressure1_train, pressure1_val = pressures_same_time_1_flatten[train_idx], pressures_same_time_1_flatten[val_idx]
surge1_train, surge1_val = surge1_input_train_flatten[train_idx], surge1_input_train_flatten[val_idx]
t_surge1_train, t_surge1_val = hours_in_year_surge_1_train_flatten[train_idx], hours_in_year_surge_1_train_flatten[val_idx]
Y_1_train, Y_1_val = Y_1_flatten[train_idx], Y_1_flatten[val_idx]

pressure2_train, pressure2_val = pressures_same_time_2_flatten[train_idx], pressures_same_time_2_flatten[val_idx]
surge2_train, surge2_val = surge2_input_train_flatten[train_idx], surge2_input_train_flatten[val_idx]
t_surge2_train, t_surge2_val = hours_in_year_surge_2_train_flatten[train_idx], hours_in_year_surge_2_train_flatten[val_idx]
Y_2_train, Y_2_val = Y_2_flatten[train_idx], Y_2_flatten[val_idx]

mean_pressures_same_time_1_train, mean_pressures_same_time_1_val = mean_pressures_same_time_1_flatten[train_idx], mean_pressures_same_time_1_flatten[val_idx]
std_pressures_same_time_1_train, std_pressures_same_time_1_val = std_pressures_same_time_1_flatten[train_idx], std_pressures_same_time_1_flatten[val_idx]
mean_pressures_same_time_2_train, mean_pressures_same_time_2_val = mean_pressures_same_time_2_flatten[train_idx], mean_pressures_same_time_2_flatten[train_idx]
std_pressures_same_time_2_train, std_pressures_same_time_2_val = std_pressures_same_time_2_flatten[train_idx], std_pressures_same_time_2_flatten[val_idx]

mean_surge1_input_train_train, mean_surge1_input_train_val = mean_surge1_input_train_flatten[train_idx], mean_surge1_input_train_flatten[val_idx]
std_surge1_input_train_train, std_surge1_input_train_val = std_surge1_input_train_flatten[train_idx], std_surge1_input_train_flatten[val_idx]
mean_surge2_input_train_train, mean_surge2_input_train_val = mean_surge2_input_train_flatten[train_idx], mean_surge2_input_train_flatten[train_idx]
std_surge2_input_train_train, std_surge2_input_train_val = std_surge2_input_train_flatten[train_idx], std_surge2_input_train_flatten[val_idx]

train_data = list(zip(
    pressure1_train, 
    pressure2_train, 
    t_surge1_train, t_surge2_train, 
    surge1_train, surge2_train, 
    mean_surge1_input_train_train, mean_surge2_input_train_train, 
    std_surge1_input_train_train, std_surge2_input_train_train,
    mean_pressures_same_time_1_train, mean_pressures_same_time_2_train,
    std_pressures_same_time_1_train, std_pressures_same_time_2_train,
    Y_1_train,
    Y_2_train
))
val_data = list(zip(
    pressure1_val, 
    pressure2_val, 
    t_surge1_val, t_surge2_val, 
    surge1_val, surge2_val, 
    mean_surge1_input_train_val, mean_surge2_input_train_val, 
    std_surge1_input_train_val, std_surge2_input_train_val,
    mean_pressures_same_time_1_val, mean_pressures_same_time_2_val,
    std_pressures_same_time_1_val, std_pressures_same_time_2_val,
    Y_1_val,
    Y_2_val
))

batch_size = 8

train_dataloader_small = DataLoader(
    train_data,
    batch_size=batch_size,
    shuffle=True
)

val_dataloader_small = DataLoader(
    val_data,
    batch_size=batch_size,
    shuffle=False
)

In [13]:
modelseqsimple = models.EncoderSeqVit()

epochs = 10
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(modelseqsimple.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)

In [14]:
for epoch in range(epochs):
    modelseqsimple.train()
    for x in tqdm(train_dataloader_small, total = len(train_dataloader_small), leave=False):
        y2 = x[-1].type(torch.FloatTensor)
        y1 = x[-2].type(torch.FloatTensor) # y = x[-1].to(device).type(torch.FloatTensor)
        xx = []
        for inp in x[:-2]:
            xx.append(inp.type(torch.FloatTensor))
        xx = tuple(xx)
        optimizer.zero_grad()
        pred = modelseqsimple(xx)
        y = torch.concat([y1, y2], dim=1)
        loss = criterion(pred, y)
        loss.backward()
        optimizer.step()
    scheduler.step()
    modelseqsimple.eval()
    val_loss = 0
    with torch.no_grad():
        for x1, x2, x3, y in tqdm(val_dataloader_small, total = len(val_dataloader_small), leave = False):
            # x1, x2, x3, y = x1.to(device), x2.to(device), x3.to(device), y.to(device)
            y2 = x[-1].type(torch.FloatTensor)
            y1 = x[-2].type(torch.FloatTensor) # y = x[-1].to(device).type(torch.FloatTensor)
            xx = []
            for inp in x[:-2]:
                xx.append(inp.type(torch.FloatTensor))
            xx = tuple(xx)
            pred = modelseqsimple(xx)
            y = torch.concat([y1, y2], dim=1)
            loss = criterion(pred, y)
            val_loss += loss.item()
    val_loss /= (len(val_dataloader_small)*batch_size)
    print(f'Epoch {epoch+1}: Validation Loss = {val_loss}')

  0%|          | 0/23 [00:00<?, ?it/s]

hidden shape torch.Size([8, 512])
mean shape torch.Size([8])
std shape torch.Size([8])
hidden shape torch.Size([8, 512])
mean shape torch.Size([8])
std shape torch.Size([8])


KeyboardInterrupt: 

In [ ]:
a = [1, 2,3,4,5,6,7,8,9]
a[-4:]

[6, 7, 8, 9]

In [ ]:
a[:-4]

[1, 2, 3, 4, 5]